In [5]:
import csv
import pandas as pd
import numpy as np
import time
import math
print('Done')

Done


In [6]:
tweets=[]
start= time.time()
train_data= pd.read_csv('data/training.csv',names = ['Type' , 'id', 'date', 'query','user', 'tweet'], encoding='latin-1')
test_data= pd.read_csv('data/test.csv',names = ['Type' , 'id', 'date', 'query','user', 'tweet'], encoding='latin-1')
test_data = test_data.drop(['id','date','query','user'], axis=1)
train_data= train_data.drop(['id','date','query','user'], axis=1)
test_data = test_data.drop(test_data[test_data['Type']==2].index)
# print(train_data[:5])
# print(train_data[:5])
print('time',time.time()-start)
print(train_data.shape)
print(test_data.shape)

time 5.489812612533569
(1600000, 2)
(359, 2)


In [40]:
tweets = train_data.set_index('Type')
tweets['words']= tweets['tweet'].str.replace(',',' ').str.replace('.', ' ').str.split()
tweets['word_count']= tweets['words'].str.len()
tweets = tweets.drop(['tweet'], axis=1)

# tweets_test = test_data.set_index('Type')
tweets_test=test_data
tweets_test['words']= tweets_test['tweet'].str.replace(',',' ').str.replace('.', ' ').str.split()
tweets_test = tweets_test.set_index('Type')
tweets_test['word_count']= tweets_test['words'].str.len()
tweets_test = tweets_test.drop(['tweet'], axis=1)
print(tweets.shape)
print(tweets_test.shape)
print(tweets[:5])

(1600000, 2)
(359, 2)
                                                  words  word_count
Type                                                               
0     [@switchfoot, http://twitpic, com/2y1zl, -, Aw...          20
0     [is, upset, that, he, can't, update, his, Face...          21
0     [@Kenichan, I, dived, many, times, for, the, b...          18
0     [my, whole, body, feels, itchy, and, like, its...          10
0     [@nationwideclass, no, it's, not, behaving, at...          21


In [41]:
def create_vocab(tweets):
    vocab = {}
    for l_words in tweets['words']:
        for word in l_words:
            vocab[word] =1
    return vocab

In [42]:
def create_dict(tweets,vocab):
    tweets_neg=tweets.loc[[0]]
    tweets_pos=tweets.loc[[4]]
    
    phi_neg = (tweets_neg.shape[0]+1)/(tweets.shape[0] +2)
    phi_pos =(tweets_pos.shape[0]+1)/(tweets.shape[0]+2)
        
    dict_neg_words={}
    dict_neg_words=vocab.copy()
    
    for l_words in tweets_neg['words']:
        for word in l_words:
                dict_neg_words[word] +=1
    
    dict_pos_words={}
    dict_pos_words=vocab.copy()
    for l_words in tweets_pos['words']:
        for word in l_words:
            dict_pos_words[word] +=1
    return dict_neg_words, dict_pos_words, phi_neg, phi_pos

In [43]:
print(12040935-1078212)
11423286-1078212

10962723


10345074

In [44]:
def check_accuracy(tweets):
    tweets_neg=tweets.loc[[0]]
    tweets_pos=tweets.loc[[4]]
    count=0
    correct =0
    total_words_neg = tweets_neg['word_count'].sum()
    total_words_pos = tweets_pos['word_count'].sum()
    print(total_words_neg)
    print(total_words_pos)
    total_words_vocab = len(vocab)
    for tweet in tweets_neg['words']:
        theta_j_neg =0
        theta_j_pos =0
        for word in tweet:
            if word in dict_neg_words:
                theta_j_neg += math.log(dict_neg_words[word])
            if word in dict_pos_words:  
                theta_j_pos += math.log(dict_pos_words[word])
        theta_j_neg -= (len(tweet) *math.log(total_words_neg))    
        theta_j_pos -= (len(tweet) *math.log(total_words_pos))
        prob_y_neg = theta_j_neg + math.log(phi_neg)
        prob_y_pos = theta_j_pos + math.log(phi_pos)
        if(prob_y_neg > prob_y_pos):
            correct += 1
    print(correct) 
    correct_neg = correct
    for tweet in tweets_pos['words']:
        theta_j_neg =0
        theta_j_pos =0
        for word in tweet:
            if word in dict_neg_words:
                theta_j_neg += math.log(dict_neg_words[word])
            if word in dict_pos_words:
                theta_j_pos += math.log(dict_pos_words[word])
        theta_j_neg -= (len(tweet) *math.log(total_words_neg))    
        theta_j_pos -= (len(tweet) *math.log(total_words_pos))
        prob_y_neg = theta_j_neg + math.log(phi_neg)
        prob_y_pos = theta_j_pos + math.log(phi_pos)
        if(prob_y_neg < prob_y_pos):
            correct += 1   
    correct_pos = correct- correct_neg       
    print(correct_pos)
    print(correct/tweets.shape[0])
    return correct_neg, correct_pos
    

In [45]:
start= time.time()
vocab = create_vocab(tweets)
print(len(vocab))
dict_neg_words, dict_pos_words, phi_neg, phi_pos= create_dict(tweets,vocab)
print(time.time()-start)
print('Done')

1078262
7.92600417137146
Done


In [46]:
start = time.time()
check_accuracy(tweets)
print('time', time.time()- start)
print('Done')   

10962673
10345024
708204
652067
0.850169375
time 16.427900075912476
Done


In [51]:
total_neg_test = tweets_test.loc[[0]].shape[0]
total_pos_test = tweets_test.loc[[4]].shape[0]
start = time.time()
correct_neg, correct_pos = check_accuracy(tweets_test)
print(time.time()-start)

2694
2633
150
141
0.8105849582172702
0.006552696228027344


In [52]:
#confusion matrix
wrong_neg = total_neg_test-correct_neg
wrong_pos = total_pos_test-correct_pos
con_mat = pd.DataFrame([[correct_neg, wrong_pos],[wrong_neg, correct_pos]], columns=['neg_actual', 'pos_actual'], index=['neg_pred', 'pos_pred'])

In [53]:
print(con_mat)

          neg_actual  pos_actual
neg_pred         150          41
pos_pred          27         141


1.(d) stop word removal

In [11]:
from nltk.tokenize import word_tokenize
from nltk.corpus import stopwords
from nltk.stem import PorterStemmer
stemming = PorterStemmer()
print('Done')

Done


In [12]:
stop_words = set(stopwords.words('english'))

In [ ]:
# t = train_data
# print(t.shape[0])
# start = time.time()
# t['words']=(t['tweet'].str.lower()).map(word_tokenize)
# # tweets['word_count']= tweets['words'].str.len()
# # tweets = tweets.drop(['tweet'], axis=1)
# print(time.time()- start)
# print(t[:12])
# print('Done')
# t = t.drop(['tweet'], axis=1)
# t.to_pickle('df_tweet')

In [15]:
def sep(str):
    temp= str.lower().replace(',',' ').replace('.',' ').split()
    res =[stemming.stem(word) for word in temp if word not in stop_words and '@' not in word]        
    return res        

In [16]:
tweets = train_data.set_index('Type')
start = time.time()
tweets['words']= tweets['tweet'].map(sep)
tweets['word_count']= tweets['words'].str.len()
tweets = tweets.drop(['tweet'], axis=1)
print(time.time()-start)
print('Done')
print(tweets[:5])
print('Done')

219.66432762145996
Done
                                                  words  word_count
Type                                                               
0     [http://twitpic, com/2y1zl, -, awww, that', bu...          13
0     [upset, can't, updat, facebook, text, might, c...          12
0     [dive, mani, time, ball, manag, save, 50%, res...          10
0                [whole, bodi, feel, itchi, like, fire]           6
0                  [behav, i'm, mad, here?, can't, see]           6
Done


In [17]:
vocab = create_vocab(tweets)
dict_neg_words, dict_pos_words, phi_neg, phi_pos= create_dict(tweets)
start = time.time()
check_accuracy(tweets)
print('time', time.time()- start)
print('Done')        

6138839
5947023
663939
626842
0.806738125
time 11.838244199752808
Done


In [18]:
start = time.time()
correct_neg, correct_pos = check_accuracy(tweets_test)
print(time.time()-start)

2694
2633
116
145
0.7270194986072424
0.0115966796875


# (e) TF-IDF

In [7]:
from sklearn.feature_extraction.text import TfidfVectorizer 
from sklearn.naive_bayes import GaussianNB

In [85]:
# train_data['tweet'].tolist()

In [8]:
tfidfVectorizer=TfidfVectorizer(use_idf=True)
 
tfidf_vectors=tfidfVectorizer.fit_transform(train_data['tweet'].tolist())

print('Done')

Done


In [9]:
print(tfidf_vectors.shape)
# print(tfidf_vectors)

(1600000, 684358)


In [8]:
# first_vector=tfidf_vectors[0]
# df = pd.DataFrame(first_vector.T.todense(), index=tfidfVectorizer.get_feature_names(), columns=["tfidf"])
# df.sort_values(by=["tfidf"])
# print(df)

In [10]:
# len(train_data['Type'].toarray().tolist())

In [ ]:
nb_model=GaussianNB()
# temp = tfidf_vectors.tocoo()
# print(temp.shape)
# t = [temp.col.tolist(), temp.data.tolist()]
# print(len(t))
nb_model.fit(tfidf_vectors.todense(),train_data['Type'].tolist())